In [ ]:
#Import data
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

df = pd.read_csv("medical_data_visualizer.csv")
# Add 'overweight' column

df['overweight'] = df['weight']/pow(df['height']/100,2)
df['overweight'].values[df['overweight'].values <= 25] = 0
df['overweight'].values[df['overweight'].values > 25] = 1
df['overweight']=df['overweight'].astype('int64')

## Normalize data by making 0 always good and 1 always bad. 
##If the value of 'cholesterol' or 'gluc' is 1, make the value 0. If the value is more than 1, make the value 1.
df['cholesterol'].values[df['cholesterol'].values == 1] = 0
df['cholesterol'].values[df['cholesterol'].values > 1] = 1
df['gluc'].values[df['gluc'].values == 1] = 0
df['gluc'].values[df['gluc'].values > 1] = 1

#Convert the data into long format and create a chart that shows the value counts of the categorical features
#using seaborn's catplot(). The dataset should be split by 'Cardio' 
#so there is one chart for each cardio value. The chart should look like examples/Figure_1.png.
# Draw Categorical Plot
def draw_cat_plot():
    # Create DataFrame for cat plot using `pd.melt` using just the values from 'cholesterol', 'gluc', 
    #'smoke', 'alco', 'active', and 'overweight'.
    df_cat=pd.melt(df,id_vars=["cardio"], value_vars =['active', 'alco','cholesterol','gluc','overweight','smoke'])


    # Group and reformat the data to split it by 'cardio'. Show the counts of each feature. You will 
    #have to rename one of the columns for the catplot to work correctly.
    df_cat=df_cat.groupby(['cardio','variable'],as_index=False)
    df_cat=df_cat[['value']].value_counts()


    # Draw the catplot with 'sns.catplot()'
    fig=sns.catplot(data=df_cat, kind='bar', x='variable', y='count',hue='value',col='cardio')
    fig.set_ylabels('total')
    fig.set_xlabels('variable')
    fig = fig.fig
    fig.savefig('catplot.png')
    return fig

   
# Draw Heat Map
def draw_heat_map():
    # Clean the data
    df_heat = df[(df['ap_lo'] <= df['ap_hi']) & (df['height'] >= df['height'].quantile(0.025)) & (df['height'] <= df['height'].quantile(0.975)) & (df['weight'] >= df['weight'].quantile(0.025)) & (df['weight'] <= df['weight'].quantile(0.975))]

    # Calculate the correlation matrix
    corr=df_heat.corr().round(1)

    # Generate a mask for the upper triangle
    mask = np.triu(np.ones_like(corr))



    # Set up the matplotlib figure
    fig, ax = plt.subplots(figsize=(16,9))

    cbar_kws = {"orientation":"vertical", 
            "shrink":0.5,
            "ticks":np.linspace(-0.08,0.24,5)
           }
    # Draw the heatmap with 'sns.heatmap()'
    sns.heatmap(corr,center=0,
                mask=mask,annot=True,square=True,
                fmt=".1f",vmax=0.30,linewidths=1,
                linecolor='white',cbar_kws=cbar_kws)

    
    # Do not modify the next two lines
    fig.savefig('heatmap.png')
    return fig